# Config

In [42]:
#editamos el path para poder accedera las funciones con la estructura actual de carpetas
from sys import path
import os
path.append(os.path.realpath('../'))

In [43]:
#importamos las funciones que hemos creado nosotros
from utils import functions as func

In [44]:
#importamos librería
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder,MinMaxScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer

import folium
import plotly.express as px
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [45]:
# Leemos los ficheros
ruta = '../data/'

df_calendar = pd.read_csv(ruta+'daily_calendar_with_events.csv')
df_sales = pd.read_csv(ruta+'item_sales.csv')
df_prices = pd.read_csv(ruta+'item_prices.csv')

# Datasets info

## Calendar

In [46]:
df_calendar.head()

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,NaN
1,2011-01-30,Sunday,2,d_2,NaN
2,2011-01-31,Monday,3,d_3,NaN
3,2011-02-01,Tuesday,4,d_4,NaN
4,2011-02-02,Wednesday,5,d_5,NaN


In [47]:
df_calendar.value_counts()

date        weekday    weekday_int  d       event         
2011-02-06  Sunday     2            d_9     SuperBowl         1
2011-08-01  Monday     3            d_185   Ramadan starts    1
2016-02-07  Sunday     2            d_1836  SuperBowl         1
2016-01-01  Friday     7            d_1799  NewYear           1
2015-11-26  Thursday   6            d_1763  Thanksgiving      1
2015-06-18  Thursday   6            d_1602  Ramadan starts    1
2015-04-05  Sunday     2            d_1528  Easter            1
2015-02-01  Sunday     2            d_1465  SuperBowl         1
2015-01-01  Thursday   6            d_1434  NewYear           1
2014-11-27  Thursday   6            d_1399  Thanksgiving      1
2014-06-29  Sunday     2            d_1248  Ramadan starts    1
2014-04-20  Sunday     2            d_1178  Easter            1
2014-02-02  Sunday     2            d_1101  SuperBowl         1
2014-01-01  Wednesday  5            d_1069  NewYear           1
2013-11-28  Thursday   6            d_1035  T

In [48]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         1913 non-null   object
 1   weekday      1913 non-null   object
 2   weekday_int  1913 non-null   int64 
 3   d            1913 non-null   object
 4   event        26 non-null     object
dtypes: int64(1), object(4)
memory usage: 74.9+ KB


In [49]:
df_calendar['date'] = pd.to_datetime(df_calendar['date'], errors='coerce')

In [50]:
df_calendar.head()

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,NaN
1,2011-01-30,Sunday,2,d_2,NaN
2,2011-01-31,Monday,3,d_3,NaN
3,2011-02-01,Tuesday,4,d_4,NaN
4,2011-02-02,Wednesday,5,d_5,NaN


In [51]:
df_calendar["week"]=df_calendar["date"].dt.isocalendar().week
df_calendar["month"]=df_calendar["date"].dt.month
df_calendar["year"]=df_calendar["date"].dt.year
df_calendar["yearweek"]=df_calendar["week"]+(df_calendar["year"]*100)

In [52]:
df_calendar["yearweek"].describe()

count          1913.0
mean     201346.78045
std        151.797076
min          201104.0
25%          201221.0
50%          201337.0
75%          201501.0
max          201653.0
Name: yearweek, dtype: Float64

## Sales

In [53]:
df_sales.head()

,id,item,category,department,store,store_code,region,d_1,d_2,d_3,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [54]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1920 entries, id to d_1913
dtypes: int64(1913), object(7)
memory usage: 446.6+ MB


Pasamos las columnas dia a fila

In [55]:
#Pasamos las columas d a filas par poder trabajar con ellas
# Cogemos las columnas.
columnas = df_sales.columns
list_id_vars = columnas[:7]   # son las columnas que necesitamos mantener.
list_value_vars = columnas[7:]

# Codigo para trasponer columnas a filas.
df_daily_sales = pd.melt(df_sales, id_vars=list_id_vars,
                   value_vars=list_value_vars ,
                   var_name='d', value_name='qty_sold')

In [56]:
df_daily_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 9 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   id          object
 1   item        object
 2   category    object
 3   department  object
 4   store       object
 5   store_code  object
 6   region      object
 7   d           object
 8   qty_sold    int64 
dtypes: int64(1), object(8)
memory usage: 3.9+ GB


## Prices

In [57]:
df_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6965706 entries, 0 to 6965705
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   item        object 
 1   category    object 
 2   store_code  object 
 3   yearweek    float64
 4   sell_price  float64
dtypes: float64(2), object(3)
memory usage: 265.7+ MB


In [58]:
df_prices.head()

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328.0,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329.0,12.7414
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330.0,10.9858
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201331.0,10.9858
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201332.0,10.9858


In [59]:
df_prices.isnull().sum()

item               0
category           0
store_code         0
yearweek      243920
sell_price         0
dtype: int64

In [60]:
df_prices=df_prices.dropna(subset=["yearweek"])
df_prices.isnull().sum()

item          0
category      0
store_code    0
yearweek      0
sell_price    0
dtype: int64

In [61]:
# Pasamos la variable yearweek a modo fecha:
df_prices["yearweek"] = df_prices["yearweek"].astype("Float64")

In [62]:
# Para hacer merge con el df_prices creamos columna d que es item+store_code dentro de prices
df_prices["id"]= df_prices["item"]+"_"+df_prices['store_code']

In [63]:
df_prices.head()

,item,category,store_code,yearweek,sell_price,id
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328.0,12.7414,ACCESORIES_1_001_NYC_1
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329.0,12.7414,ACCESORIES_1_001_NYC_1
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330.0,10.9858,ACCESORIES_1_001_NYC_1
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201331.0,10.9858,ACCESORIES_1_001_NYC_1
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201332.0,10.9858,ACCESORIES_1_001_NYC_1


In [64]:
del df_prices["item"], df_prices['store_code'], df_prices['category']

In [65]:
df_prices["yearweek"].describe()

count        6721786.0
mean     201382.418346
std         145.018929
min           201105.0
25%           201248.0
50%           201410.0
75%           201515.0
max           201617.0
Name: yearweek, dtype: Float64

### Imputacion nulos en sellprice

In [66]:
#generamos las combinaciones posibles de semana y id, para luego generar las observaciones con nulos 
all_combinations = pd.MultiIndex.from_product([df_prices["id"].unique(), df_prices["yearweek"].unique()], names=['id', 'yearweek'])
df_yearweek = pd.DataFrame(index=all_combinations).reset_index()

In [67]:
print('df_yearweek')
print('shape:')
print(df_yearweek.shape)
print('unique:')
print(df_yearweek.nunique())
print()
print('df_prices')
print('shape:')
print(df_prices.shape)
print('unique:')
print(df_prices.nunique())

df_yearweek
shape:
(8506710, 2)
unique:
id          30490
yearweek      279
dtype: int64

df_prices
shape:
(6721786, 3)
unique:
yearweek        279
sell_price     1860
id            30490
dtype: int64


In [68]:
df_prices = pd.merge(df_yearweek, df_prices, on=['id', 'yearweek'], how='left')

In [69]:
print('df_prices')
print('shape:')
print(df_prices.shape)
print('unique:')
print(df_prices.nunique())
print('nulos')
df_prices.isnull().sum()

df_prices
shape:
(8506710, 3)
unique:
id            30490
yearweek        279
sell_price     1860
dtype: int64
nulos


id                  0
yearweek            0
sell_price    1784924
dtype: int64

In [70]:
df_prices.sort_values(['id', 'yearweek'], ascending = True, inplace = True)

Para ver si hacemos bien la imputacion seleccionamos algun id con nulos para comprobar

In [84]:
rows_with_null_sell_price = df_prices[df_prices['sell_price'].isnull()]

# Seleccionar un ID que tenga al menos una semana con 'sell_price' nulo
selected_id = rows_with_null_sell_price['id'].sample(n=1).iloc[0]

# Mostrar el ID seleccionado
print(f"ID seleccionado: {selected_id}")

ID seleccionado: ACCESORIES_2_079_BOS_3


In [85]:
sample_ts = df_prices[df_prices['id']=='ACCESORIES_2_079_BOS_3']

In [40]:
df_prices["sell_price"] = df_prices.groupby("id", group_keys = False)["sell_price"].apply(
    lambda series: series.backfill().ffill())

# Union datasets

## Calendar sales 
Realizamos la union del df_calendar y el df_daily sales a través de la variable d.

In [23]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         1913 non-null   datetime64[ns]
 1   weekday      1913 non-null   object        
 2   weekday_int  1913 non-null   int64         
 3   d            1913 non-null   object        
 4   event        26 non-null     object        
 5   week         1913 non-null   UInt32        
 6   month        1913 non-null   int32         
 7   year         1913 non-null   int32         
 8   yearweek     1913 non-null   Int64         
dtypes: Int64(1), UInt32(1), datetime64[ns](1), int32(2), int64(1), object(3)
memory usage: 116.0+ KB


In [44]:
df_calendar_sales = pd.merge(df_daily_sales,df_calendar[['date','d', 'yearweek']], on='d', how='inner')

Aplicamos resample para obtener las ventas por "semana". Se observa que del 2016 tenemos pocos meses, igual que 2011. Ojo hemos convertido "date" en índice.

In [45]:
df_calendar_sales.head()

,id,item,category,department,store,store_code,region,d,qty_sold,date,yearweek
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,201104
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,201104
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,201104
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,201104
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,201104


In [46]:
df_calendar_sales.shape

(58327370, 11)

## Reducimos dimensionalidad

In [47]:
df_calendar_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 11 columns):
 #   Column      Dtype         
---  ------      -----         
 0   id          object        
 1   item        object        
 2   category    object        
 3   department  object        
 4   store       object        
 5   store_code  object        
 6   region      object        
 7   d           object        
 8   qty_sold    int64         
 9   date        datetime64[ns]
 10  yearweek    Int64         
dtypes: Int64(1), datetime64[ns](1), int64(1), object(8)
memory usage: 4.8+ GB


In [50]:
df_weekly_sales = (df_calendar_sales
                   .groupby(['id', 'item', 'category', 'department', 'store', 'store_code', 'region','yearweek', pd.Grouper(key='date', freq='W-Mon')], as_index=False)
                   ['qty_sold'].sum()
                   )


- pd.Grouper es una clase en la biblioteca pandas que proporciona una interfaz flexible para realizar operaciones de agrupación en series temporales. Se utiliza comúnmente con el método groupby para agrupar datos temporales de manera conveniente y realizar operaciones de agregación en esos grupos.
- key='date': Especifica la columna por la cual se realizará la agrupación. En este caso, la columna 'date'.
- freq='W-Mon': Especifica la frecuencia con la que se agruparán los datos. En este ejemplo, se agruparán semanalmente ('W') y el lunes ('Mon') será el día de inicio de la semana.

In [51]:
df_weekly_sales.shape

(16799990, 10)

In [52]:
df_weekly_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16799990 entries, 0 to 16799989
Data columns (total 10 columns):
 #   Column      Dtype         
---  ------      -----         
 0   id          object        
 1   item        object        
 2   category    object        
 3   department  object        
 4   store       object        
 5   store_code  object        
 6   region      object        
 7   yearweek    Int64         
 8   date        datetime64[ns]
 9   qty_sold    int64         
dtypes: Int64(1), datetime64[ns](1), int64(1), object(7)
memory usage: 1.3+ GB


## Union Weekly Sales con Prices

In [53]:
df_merged_final= pd.merge(df_weekly_sales, df_prices, on=['id', "yearweek"], how='left')  

In [55]:
df_merged_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16799990 entries, 0 to 16799989
Data columns (total 11 columns):
 #   Column      Dtype         
---  ------      -----         
 0   id          object        
 1   item        object        
 2   category    object        
 3   department  object        
 4   store       object        
 5   store_code  object        
 6   region      object        
 7   yearweek    Int64         
 8   date        datetime64[ns]
 9   qty_sold    int64         
 10  sell_price  float64       
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(1), object(7)
memory usage: 1.4+ GB


In [56]:
df_merged_final.to_csv((ruta+'df_merged.csv'), index=False)